In [1]:
import pickle
import pandas as pd
import numpy as np

# 학습용 데이터 전처리(100번대 제외)

In [2]:
import glob

path = './pickled'

file_list = glob.glob(path + "\\*.csv")
df_list = []

for file in file_list:
    current_data = pd.read_csv(file)
    current_data.reset_index()
    df_list.append(current_data)

# 데이터 병합

In [3]:
train05 = pd.concat([df_list[0],df_list[1],df_list[2],df_list[3],df_list[4],df_list[5],df_list[6],df_list[7],df_list[8],df_list[9]], axis=0)
train05 = train05.groupby(train05['actor_account']).sum()
train05.drop(['actor_account'], axis=1, inplace=True)

In [4]:
# 최고레벨 수정
max_level = train05[['max_level_of_a_dayday8','max_level_of_a_dayday9','max_level_of_a_dayday10','max_level_of_a_dayday11','max_level_of_a_dayday12','max_level_of_a_dayday13','max_level_of_a_dayday14','max_level_of_a_dayday15','max_level_of_a_dayday16','max_level_of_a_dayday17']]
train05['max_level'] = max_level.max(axis=1)
train05 = train05.drop(['max_level_of_a_dayday8','max_level_of_a_dayday9','max_level_of_a_dayday10','max_level_of_a_dayday11','max_level_of_a_dayday12','max_level_of_a_dayday13','max_level_of_a_dayday14','max_level_of_a_dayday15','max_level_of_a_dayday16','max_level_of_a_dayday17'], axis=1)

In [5]:
# 동일지점 접속률
train05['same_ip_rate'] = (train05['103'] / train05['log_count']).round(4)
# 친구 삭제, 등록, 차단 count
train05['friend_action'] = train05['134'] + train05['135']
# 보조 옵션
train05['assist_option'] = train05['272'] + train05['273']  + train05['274']
# 스킬 획득 및 삭제
train05['skill_del_or_get'] = train05['401'] + train05['403']
# 스티그마스킬 획득 및 삭제
train05['stigmaskill_del_or_get'] = train05['406'] + train05['407']
# 퀘스트관련 로그
train05['quest_log_count'] = train05['501'] + train05['502'] + train05['503'] +train05['504'] + train05['505']
# 레기온관련 로그
train05['about_region'] = train05['601'] + train05['602'] + train05['603'] + train05['604']

# 중복데이터 제거
train05.drop(columns = ['134','135','272','273','274','401','403','406','407','501','502','503','504','505',
                       '601','602','603','604'], axis = 1, inplace = True)

In [6]:
train05.head()

,103,104,110,112,113,114,115,116,118,119,...,197,607,max_level,same_ip_rate,friend_action,assist_option,skill_del_or_get,stigmaskill_del_or_get,quest_log_count,about_region
actor_account,,,,,,,,,,,,,,,,,,,,,
1000115,11,10,0,0,2,0,0,0,52,51,...,0.0,0.0,43.0,1.0000,0,0,37,40,725,0.0
1001198,21,21,0,0,0,0,0,0,0,0,...,0.0,0.0,10.0,1.4000,0,0,0,0,0,0.0
1001279,9,9,0,0,1,0,0,0,28,28,...,0.0,0.0,50.0,1.5000,0,0,25,42,56,0.0
1002212,10,10,1,1,14,0,2,0,11,11,...,0.0,0.0,12.0,1.2500,4,0,124,0,477,0.0
1002538,8,8,0,0,2,0,0,6,22,22,...,0.0,0.0,23.0,1.3333,0,0,26,3,436,0.0


In [30]:
label = pd.read_csv('./labeled_account.csv')
label = label.rename({'account':'actor_account'}, axis = 1)

In [33]:
data = pd.merge(train05, label, on = 'actor_account')

In [8]:
train05.to_csv('TEST.csv', index = True)